## Train

In [ ]:
%%capture
!pip install datasets evaluate transformers[sentencepiece]
!pip install rouge_score

In [ ]:
!gsutil cp -r gs://vietai_public/viT5/data/vietnews .
!gsutil cp -r gs://vietai_public/viT5/data/wikilingua .


Copying gs://vietai_public/viT5/data/vietnews/test.tsv...
Copying gs://vietai_public/viT5/data/vietnews/test_dedup.tsv...
Copying gs://vietai_public/viT5/data/vietnews/train_dedup.tsv...
| [3 files][422.4 MiB/422.4 MiB]   59.8 MiB/s                                   
Operation completed over 3 objects/422.4 MiB.                                    
Copying gs://vietai_public/viT5/data/wikilingua/test.tsv...
Copying gs://vietai_public/viT5/data/wikilingua/train.tsv...
Copying gs://vietai_public/viT5/data/wikilingua/val.tsv...
/ [3 files][ 63.5 MiB/ 63.5 MiB]                                                
Operation completed over 3 objects/63.5 MiB.                                     


In [ ]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainer, TrainingArguments, Seq2SeqTrainingArguments
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("VietAI/vit5-base")  
model = AutoModelForSeq2SeqLM.from_pretrained("VietAI/vit5-base")
model.to('cuda')

In [ ]:
def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["inputs"], max_length=1024, truncation=True, padding=True
    )
    
    labels = tokenizer(
        examples["labels"], max_length=256, truncation=True, padding=True
    )
    model_inputs['labels'] = labels['input_ids']
    model_inputs['input_ids'] = model_inputs['input_ids']
    return model_inputs

In [ ]:
input_lines = []
label_lines = []

task = 'wikilingua'
train_file = 'train.tsv'

with open(f'{task}/{train_file}') as file:
  for line in file:
    line = line.strip().split('\t')
    input_lines.append(line[0] +'</s>')
    label_lines.append(line[1])


dict_obj = {'inputs': input_lines, 'labels': label_lines}
dataset = Dataset.from_dict(dict_obj)
tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=['inputs'], num_proc=8)



#0:   0%|          | 0/2 [00:00<?, ?ba/s]

#1:   0%|          | 0/2 [00:00<?, ?ba/s]

#2:   0%|          | 0/2 [00:00<?, ?ba/s]

#3:   0%|          | 0/2 [00:00<?, ?ba/s]

#4:   0%|          | 0/2 [00:00<?, ?ba/s]

#5:   0%|          | 0/2 [00:00<?, ?ba/s]

#6:   0%|          | 0/2 [00:00<?, ?ba/s]

#7:   0%|          | 0/2 [00:00<?, ?ba/s]

#8:   0%|          | 0/2 [00:00<?, ?ba/s]

#9:   0%|          | 0/2 [00:00<?, ?ba/s]

In [ ]:

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="pt")


training_args = Seq2SeqTrainingArguments("tmp/",
                                      do_train=True,
                                      do_eval=False,
                                      num_train_epochs=30,
                                      learning_rate=1e-5,
                                      warmup_ratio=0.05,
                                      weight_decay=0.01,
                                      per_device_train_batch_size=4,
                                      per_device_eval_batch_size=4,
                                      logging_dir='./log',
                                      group_by_length=True,
                                      save_strategy="epoch",
                                      save_total_limit=3,
                                      #eval_steps=1,
                                      #evaluation_strategy="steps",
                                      # evaluation_strategy="no",
                                      fp16=True,
                                      )


# AdaFactor for ViT5-large models as it based on T5v1.1.
# See https://medium.com/the-artificial-impostor/paper-adafactor-adaptive-learning-rates-with-sublinear-memory-cost-a543abffa37
# 
# from transformers.optimization import Adafactor, AdafactorSchedule
# optimizer = Adafactor(
#     model.parameters(),
#     lr=1e-3,
#     eps=(1e-30, 1e-3),
#     clip_threshold=1.0,
#     decay_rate=-0.8,
#     beta1=None,
#     weight_decay=0.0,
#     relative_step=False,
#     scale_parameter=False,
#     warmup_init=False
# )
# lr_scheduler = AdafactorSchedule(optimizer)


In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    data_collator=data_collator,
)

trainer.train()

## Inference

In [ ]:
from datasets import load_metric
metric = load_metric("rouge")


In [ ]:
input_lines = []
label_lines = []
with open(f'{task}/test.tsv') as file:
  for line in file:
    line = line.strip().split('\t')
    input = line[0]
    input_lines.append(input +'</s>')
    label_lines.append(line[1])



input_lines  = input_lines
label_lines = label_lines
dict_obj = {'inputs': input_lines, 'labels': label_lines}

dataset = Dataset.from_dict(dict_obj)
test_tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=['inputs'], num_proc=10)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="pt")

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#4:   0%|          | 0/1 [00:00<?, ?ba/s]

#5:   0%|          | 0/1 [00:00<?, ?ba/s]

#6:   0%|          | 0/1 [00:00<?, ?ba/s]

#7:   0%|          | 0/1 [00:00<?, ?ba/s]

#8:   0%|          | 0/1 [00:00<?, ?ba/s]

#9:   0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("/content/tmp/checkpoint-85675")
model.to('cuda')

loading configuration file /content/tmp/checkpoint-85675/config.json
Model config T5Config {
  "_name_or_path": "/content/tmp/checkpoint-85675",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "torch_dtype": "float32",
  "transformers_version": "4.21.3",
  "use_cache": true,
  "vocab_size": 36096
}

loading weights file /content/tmp/checkpoint-85675/pytorch_model.bin
All model checkpoint weights were used when initializing T5ForConditionalGeneration.


T5ForConditionalGeneration(
  (shared): Embedding(36096, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(36096, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [ ]:
import torch 
import numpy as np
metrics = load_metric('rouge')

max_target_length = 256
dataloader = torch.utils.data.DataLoader(test_tokenized_datasets, collate_fn=data_collator, batch_size=32)

predictions = []
references = []
for i, batch in enumerate(tqdm(dataloader)):
  outputs = model.generate(
      input_ids=batch['input_ids'].to('cuda'),
      max_length=max_target_length,
      attention_mask=batch['attention_mask'].to('cuda'),
  )
  with tokenizer.as_target_tokenizer():
    outputs = [tokenizer.decode(out, clean_up_tokenization_spaces=False, skip_special_tokens=True) for out in outputs]

    labels = np.where(batch['labels'] != -100,  batch['labels'], tokenizer.pad_token_id)
    actuals = [tokenizer.decode(out, clean_up_tokenization_spaces=False, skip_special_tokens=True) for out in labels]
  predictions.extend(outputs)
  references.extend(actuals)
  metrics.add_batch(predictions=outputs, references=actuals)


metrics.compute()


  0%|          | 0/123 [00:00<?, ?it/s]

{'rouge1': AggregateScore(low=Score(precision=0.6768047009064575, recall=0.5423760911872392, fmeasure=0.5705682484836969), mid=Score(precision=0.6830006654364508, recall=0.5478652641300974, fmeasure=0.5745927701063134), high=Score(precision=0.6885843534466735, recall=0.552987118898428, fmeasure=0.578540941220134)),
 'rouge2': AggregateScore(low=Score(precision=0.3607137448492059, recall=0.28528514315021286, fmeasure=0.300617455896622), mid=Score(precision=0.36567725713182586, recall=0.28971013689007435, fmeasure=0.30471236408017977), high=Score(precision=0.3706997500164587, recall=0.29446569108896653, fmeasure=0.3088312951895017)),
 'rougeL': AggregateScore(low=Score(precision=0.48795067760878647, recall=0.38956360151268665, fmeasure=0.4088891966544668), mid=Score(precision=0.49308459719270403, recall=0.3940464554437201, fmeasure=0.4125321080267584), high=Score(precision=0.4983781504616266, recall=0.3983343291280753, fmeasure=0.4160949347351831)),
 'rougeLsum': AggregateScore(low=Score

In [ ]:
[{k: v.mid.fmeasure} for k,v in metrics.compute(predictions=predictions, references=references).items()]


[{'rouge1': 0.5745927701063134},
 {'rouge2': 0.30471236408017977},
 {'rougeL': 0.4125321080267584},
 {'rougeLsum': 0.4125279206347704}]